## Import libraries, set path and load the face detector

In [ ]:
# Connect with google drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
import glob
from string import digits
import cv2
import dlib
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

In [ ]:
# tensorflow & keras version
print(f'TensorFlow version: {tf.__version__}')
print(f'Keras version: {keras.__version__}')

TensorFlow version: 2.9.2
Keras version: 2.9.0


In [ ]:
path = '/content/drive/MyDrive/spiced_projects/final_project/model_V3/'
remove_digits = str.maketrans('', '', digits)

In [ ]:
# Download Dlib CNN face detector
#! wget http://dlib.net/files/mmod_human_face_detector.dat.bz2
#!bzip2 -dk mmod_human_face_detector.dat.bz2
#%rm mmod_human_face_detector.dat.bz2

# load the CNN face detector
path_facedetector = path + 'mmod_human_face_detector.dat'
dnnFaceDetector = dlib.cnn_face_detection_model_v1(path_facedetector)

##Preprocess the training data

In [ ]:
train_image_path_names = []
class_names = set()
for file_name in glob.glob(path + 'data/train_images/*[1-9]*.jpeg'):
  train_image_path_names.append(file_name)
  class_names.add(train_image_path_names[-1].split('/')[-1].translate(remove_digits).split('.jpeg')[0])

In [ ]:
print(class_names)
print(len(train_image_path_names))

{'toddlerface', 'adultface'}
245


In [ ]:
isExist = os.path.exists(path + 'data/train_images_crop/')
if not isExist:
   os.mkdir(path + 'data/train_images_crop/')
   print("The new directory " + path + 'data/train_images_crop/' +  " is created!")
else:
  print("The directory " + path + 'data/train_images_crop/' + " was already created!")

The directory /content/drive/MyDrive/spiced_projects/final_project/model_V3/data/train_images_crop/ was already created!


In [ ]:
for class_ in class_names:
  isExist = os.path.exists(path + 'data/train_images_crop/' + class_ + '/')
  if not isExist:
    os.mkdir(path + 'data/train_images_crop/' + class_ + '/')
    print("The new directory " + path + 'data/train_images_crop/' + class_ + " is created!")
  else:
    print("The directory " + path + 'data/train_images_crop/' + class_ + " was already created!")

The new directory /content/drive/MyDrive/spiced_projects/final_project/model_V3/data/train_images_crop/toddlerface is created!
The new directory /content/drive/MyDrive/spiced_projects/final_project/model_V3/data/train_images_crop/adultface is created!


In [ ]:
for file_name in train_image_path_names:
  img = cv2.imread(file_name)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # the dnnFaceDetector crops the faces in the images, that we will need for the training 
  rects = dnnFaceDetector(gray, 1)
  left,top,right,bottom = 0,0,0,0
  for (i,rect) in enumerate(rects):
    left   = rect.rect.left()     #x1
    top    = rect.rect.top()      #y1
    right  = rect.rect.right()    #x2
    bottom = rect.rect.bottom()   #y2
  width  = right-left
  height = bottom-top
  img_crop = img[top:top+height,left:left+width]
  if file_name.find("adult") != -1:
    folder_name = 'adultface'
  else:
    folder_name = 'toddlerface'
  img_path = path + 'data/train_images_crop/' + folder_name + '/' + file_name.split('/')[-1].split('_')[0] 
  print('img_path: ', img_path)
  try:
    cv2.imwrite(img_path,img_crop)
  except:
    pass

##Preprocess the test data

In [ ]:
test_image_path_names = []
class_names = set()
for file_name in glob.glob(path + 'data/test_images/*[1-9]*.jpeg'):
  test_image_path_names.append(file_name)
  class_names.add(test_image_path_names[-1].split('/')[-1].translate(remove_digits).split('.jpeg')[0])

In [ ]:
print(class_names)
print(len(test_image_path_names))

{'toddlerface', 'adultface'}
87


In [ ]:
isExist = os.path.exists(path + 'data/test_images_crop/')
if not isExist:
   os.mkdir(path + 'data/test_images_crop/')
   print("The new directory " + path + 'data/test_images_crop/' + " is created!")
else:
  print("The directory " + path + 'data/test_images_crop/' + " was already created!")

The new directory /content/drive/MyDrive/spiced_projects/final_project/model_V3/data/test_images_crop/ is created!


In [ ]:
for class_ in class_names:
  isExist = os.path.exists(path + 'data/test_images_crop/' + class_ + '/')
  if not isExist:
    os.mkdir(path + 'data/test_images_crop/' + class_ + '/')
    print("The new directory " + path + 'data/test_images_crop/' + class_ + " is created!")
  else:
    print("The directory " + path + 'data/test_images_crop/' + class_ + " was already created!")

The new directory /content/drive/MyDrive/spiced_projects/final_project/model_V3/data/test_images_crop/toddlerface is created!
The new directory /content/drive/MyDrive/spiced_projects/final_project/model_V3/data/test_images_crop/adultface is created!


In [ ]:
for file_name in test_image_path_names:
  img = cv2.imread(file_name)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # the dnnFaceDetector crops the faces in the images, that we will need for the training 
  rects = dnnFaceDetector(gray, 1)
  left,top,right,bottom = 0,0,0,0
  for (i,rect) in enumerate(rects):
    left   = rect.rect.left()     #x1
    top    = rect.rect.top()      #y1
    right  = rect.rect.right()    #x2
    bottom = rect.rect.bottom()   #y2
  width  = right-left
  height = bottom-top
  img_crop = img[top:top+height,left:left+width]
  if file_name.find("adult") != -1:
    folder_name = 'adultface'
  else:
    folder_name = 'toddlerface'
  img_path = path + 'data/test_images_crop/' + folder_name + '/' + file_name.split('/')[-1].split('_')[0] 
  print('img_path: ', img_path)
  try:
    cv2.imwrite(img_path,img_crop)
  except:
    pass

## Prepare training and test data and instantiate pre-trained model

###VGG_FACE_MODEL architecture

In [ ]:
#Use VGG_FACE_MODEL with the layer architecture: https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/ 

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('sigmoid'))

In [ ]:
# load the VGG Face model weights: https://www.kaggle.com/datasets/paraskumarsahu/vgg-face
path_vgg_face_weights = path + 'vgg_face_weights.h5'
model.load_weights(path_vgg_face_weights)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                        

In [ ]:
# the last layer needs to be removed for the vgg_face classifier 
vgg_face = Model(inputs = model.layers[0].input,outputs = model.layers[-2].output)
# vgg_face.summary()

In [ ]:
# Save model for later use
tf.keras.models.save_model(vgg_face, path + 'saved_model/vgg_face.h5')

###training data

In [ ]:
x_train = []
y_train = []
class_folders = os.listdir(path + 'data/train_images_crop/')
class_rep = dict()

for i,class_ in enumerate(class_folders):
  class_rep[i] = class_
  image_names = os.listdir(path + 'data/train_images_crop/' + class_ + '/')

  for image_name in image_names:
    img = load_img(path + 'data/train_images_crop/' + class_ + '/' + image_name ,target_size=(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img,axis = 0)  
    # The preprocess_input function is meant to adequate your image to the format the model requires.  
    img = preprocess_input(img)
    # what is line 17 actually doing?
    img_encode = vgg_face(img)
    # what is line 19 actually doing?
    x_train.append(np.squeeze(K.eval(img_encode)).tolist())
    y_train.append(i)

In [ ]:
class_rep

{0: 'toddlerface', 1: 'adultface'}

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

np.save('train_data',x_train)
np.save('train_labels',y_train)

### test data

In [ ]:
x_test = []
y_test = []
class_folders = os.listdir(path + 'data/test_images_crop/')
class_rep = dict()

for i,class_ in enumerate(class_folders):
  class_rep[i] = class_
  image_names = os.listdir(path + 'data/test_images_crop/' + class_ + '/')

  for image_name in image_names:
    img = load_img(path + 'data/test_images_crop/' + class_ + '/' + image_name ,target_size=(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img,axis = 0)  
    # The preprocess_input function is meant to adequate your image to the format the model requires.  
    img = preprocess_input(img)
    # what is line 17 actually doing?
    img_encode = vgg_face(img)
    # what is line 19 actually doing?
    x_test.append(np.squeeze(K.eval(img_encode)).tolist())
    y_test.append(i)

In [ ]:
class_rep

{0: 'toddlerface', 1: 'adultface'}

In [ ]:
x_test = np.array(x_test)
y_test = np.array(y_test)

np.save('test_data',x_test)
np.save('test_labels',y_test)

## Training time

In [ ]:
x_train=np.load('train_data.npy')
y_train=np.load('train_labels.npy')
x_test=np.load('test_data.npy')
y_test=np.load('test_labels.npy')

In [ ]:
x_train.shape[1]

2622

In [ ]:
len(x_train)

208

In [ ]:
# Sigmoid regressor to classify images based on encoding 
# https://medium.com/analytics-vidhya/face-recognition-with-vgg-face-in-keras-96e6bc1951d5

# How to choose an activation function
# https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/#:~:text=The%20hyperbolic%20tangent%20activation%20function,the%20range%20%2D1%20to%201.


#################################
######## Baseline Model #########
#################################

classifier_model = Sequential()

classifier_model.add(Dense(units = 100, input_dim = x_train.shape[1], kernel_initializer = 'glorot_uniform'))     # The goal of glorot_uniform (Xavier) Initialization is to initialize the weights such that the variance of the activations are the same across every layer. This constant variance helps prevent the gradient from exploding or vanishing.
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh')) # Hyperbolic tangent activation function.
classifier_model.add(Dropout(0.3))

classifier_model.add(Dense(units = 10,kernel_initializer = 'glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.2))

classifier_model.add(Dense(units = 10,kernel_initializer = 'glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('tanh'))
classifier_model.add(Dropout(0.1))

classifier_model.add(Dense(units = 1,kernel_initializer = 'he_uniform'))  
classifier_model.add(Activation('sigmoid'))
classifier_model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                         optimizer = 'nadam',
                         metrics = ['accuracy']
                         ) # try other optimizer
#classifier_model.compile(loss = tf.keras.losses.BinaryCrossentropy(),optimizer = 'nadam',metrics = ['accuracy']) # try other optimizer

In [ ]:
classifier_model.fit(x_train,
                     y_train,
                     epochs = 200,
                     validation_data=(x_test,
                                      y_test))

Epoch 1/200
7/7 [==============================] - 3s 54ms/step - loss: 0.4279 - accuracy: 0.8173 - val_loss: 0.2835 - val_accuracy: 0.8987
Epoch 2/200
7/7 [==============================] - 0s 12ms/step - loss: 0.2930 - accuracy: 0.9231 - val_loss: 0.2614 - val_accuracy: 0.8861
Epoch 3/200
7/7 [==============================] - 0s 15ms/step - loss: 0.2584 - accuracy: 0.9327 - val_loss: 0.2658 - val_accuracy: 0.8861
Epoch 4/200
7/7 [==============================] - 0s 12ms/step - loss: 0.2020 - accuracy: 0.9712 - val_loss: 0.2778 - val_accuracy: 0.8861
Epoch 5/200
7/7 [==============================] - 0s 12ms/step - loss: 0.2234 - accuracy: 0.9519 - val_loss: 0.2526 - val_accuracy: 0.8987
Epoch 6/200
7/7 [==============================] - 0s 12ms/step - loss: 0.1748 - accuracy: 0.9663 - val_loss: 0.2552 - val_accuracy: 0.9114
Epoch 7/200
7/7 [==============================] - 0s 14ms/step - loss: 0.2049 - accuracy: 0.9567 - val_loss: 0.2925 - val_accuracy: 0.8734
Epoch 8/200
7/7 [===

In [ ]:
# Save model for later use
tf.keras.models.save_model(classifier_model, path + 'saved_model/face_classifier_model.h5')

## Hyperparameter Tuning I

In [ ]:
! pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 6.5 MB/s 
     |████████████████████████████████| 1.6 MB 49.9 MB/s 


In [ ]:
# https://towardsdatascience.com/hyperparameter-tuning-with-kerastuner-and-tensorflow-c4a4d690b31a

import tensorflow as tf
import kerastuner as kt
from tensorflow import keras
print(f"TensorFlow Version: {tf.__version__}")
print(f"KerasTuner Version: {kt.__version__}")

TensorFlow Version: 2.9.2
KerasTuner Version: 1.0.3


In [ ]:
# Load and split data into train and test sets
X_train = x_train
Y_train = y_train
X_test  = x_test
Y_test  = y_test

# Normalize pixels to values between 0 and 1
#X_train = X_train.astype('float32') / 255.0
#X_test = X_test.astype('float32') / 255.0

In [ ]:
# Build baseline model with Sequential API

b_model = Sequential()

b_model.add(Dense(units = 100, input_dim = x_train.shape[1], kernel_initializer = 'glorot_uniform'))
b_model.add(BatchNormalization())
b_model.add(Activation('tanh')) # Hyperbolic tangent activation function.
b_model.add(Dropout(0.3))

b_model.add(Dense(units = 10,kernel_initializer = 'glorot_uniform'))
b_model.add(BatchNormalization())
b_model.add(Activation('tanh'))
b_model.add(Dropout(0.2))

b_model.add(Dense(units = 10,kernel_initializer = 'glorot_uniform'))
b_model.add(BatchNormalization())
b_model.add(Activation('tanh'))
b_model.add(Dropout(0.1))

b_model.add(Dense(units = 1,kernel_initializer = 'he_uniform'))  
b_model.add(Activation('sigmoid'))                

In [ ]:
# Set training parameters
b_model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                         optimizer = 'nadam',
                         metrics = ['accuracy'])

In [ ]:
# Number of epochs
NUM_EPOCHS = 200

# Early stopping set after 20 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# Train model
b_model.fit(X_train, Y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

Epoch 1/200
6/6 - 2s - loss: 0.5650 - accuracy: 0.7349 - val_loss: 0.5704 - val_accuracy: 0.7381 - 2s/epoch - 346ms/step
Epoch 2/200
6/6 - 0s - loss: 0.3596 - accuracy: 0.8855 - val_loss: 0.4378 - val_accuracy: 0.8810 - 65ms/epoch - 11ms/step
Epoch 3/200
6/6 - 0s - loss: 0.3155 - accuracy: 0.9157 - val_loss: 0.4262 - val_accuracy: 0.8333 - 61ms/epoch - 10ms/step
Epoch 4/200
6/6 - 0s - loss: 0.2679 - accuracy: 0.9458 - val_loss: 0.4526 - val_accuracy: 0.8333 - 65ms/epoch - 11ms/step
Epoch 5/200
6/6 - 0s - loss: 0.2694 - accuracy: 0.9337 - val_loss: 0.4799 - val_accuracy: 0.7857 - 61ms/epoch - 10ms/step
Epoch 6/200
6/6 - 0s - loss: 0.2539 - accuracy: 0.9217 - val_loss: 0.4276 - val_accuracy: 0.8333 - 61ms/epoch - 10ms/step
Epoch 7/200
6/6 - 0s - loss: 0.2432 - accuracy: 0.9337 - val_loss: 0.3815 - val_accuracy: 0.8095 - 58ms/epoch - 10ms/step
Epoch 8/200
6/6 - 0s - loss: 0.2308 - accuracy: 0.9398 - val_loss: 0.3266 - val_accuracy: 0.8571 - 79ms/epoch - 13ms/step
Epoch 9/200
6/6 - 0s - lo

In [ ]:
import pandas as pd

def evaluate_model(model, X_test, Y_test):
    """
    evaluate model on test set and show results in dataframe.
    
    Parameters
    ----------
    model : keras model
        trained keras model.
    X_test : numpy array
        Features of holdout set.
    y_test : numpy array
        Labels of holdout set.
        
    Returns
    -------
    display_df : DataFrame
        Pandas dataframe containing evaluation results.
    """
    eval_dict = model.evaluate(X_test, Y_test, return_dict=True)
    
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    return display_df

# Evaluate model on test set and add results to dataframe
results = evaluate_model(b_model, X_test, Y_test)

# Set index to 'Baseline'
results.index = ['Baseline']

# Display results
results.head()

3/3 [==============================] - 0s 5ms/step - loss: 0.2642 - accuracy: 0.8987


,loss,accuracy
Baseline,0.264184,0.898734


In [ ]:
def build_model(hp,X_train=X_train):
    """
    Builds model and sets up hyperparameter space to search.
    
    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.
        
    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    # Initialize sequential API and start building model.
    model = keras.Sequential()
    model.add(Dense(units = 100, input_dim = X_train.shape[1], kernel_initializer = 'glorot_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.3))
    
    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 6
    # Number of Units: 8 - 14 with stepsize of 1
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=8, max_value=14, step=1),
                kernel_initializer = 'glorot_uniform',
                activation="tanh")
            )
        model.add(BatchNormalization())
        # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(keras.layers.Dense(units=1, activation="sigmoid",kernel_initializer = 'he_uniform'))
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=["accuracy"])
    
    return model

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=NUM_EPOCHS,
                     factor=3,
                     hyperband_iterations=1,
                     directory="kt_dir",
                     project_name="kt_hyperband")

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 14, 'step': 1, 'sampling': None}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

tuner.search(X_train, Y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

Trial 254 Complete [00h 00m 09s]
val_accuracy: 0.9285714030265808

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 13m 17s


In [ ]:
for h_param in [f"units_{i}" for i in range(1,4)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

units_1 11
units_2 10
units_3 14
learning_rate 0.0001


In [ ]:
# Get the optimal hyperparameters from the results
best_hps = tuner.get_best_hyperparameters()[0]
best_hps

In [ ]:
# Build model
h_model = tuner.hypermodel.build(best_hps)
h_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 100)               262300    
                                                                 
 batch_normalization_7 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 100)               0         
                                                                 
 dropout_7 (Dropout)         (None, 100)               0         
                                                                 
 dense_10 (Dense)            (None, 11)                1111      
                                                                 
 batch_normalization_8 (Batc  (None, 11)               44        
 hNormalization)                                      

In [ ]:
# Train the hypertuned model
h_model.fit(X_train, Y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

Epoch 1/200
6/6 - 1s - loss: 0.7557 - accuracy: 0.5602 - val_loss: 0.7002 - val_accuracy: 0.5952 - 1s/epoch - 184ms/step
Epoch 2/200
6/6 - 0s - loss: 0.5229 - accuracy: 0.7410 - val_loss: 0.6522 - val_accuracy: 0.6190 - 64ms/epoch - 11ms/step
Epoch 3/200
6/6 - 0s - loss: 0.4688 - accuracy: 0.7952 - val_loss: 0.5789 - val_accuracy: 0.7857 - 48ms/epoch - 8ms/step
Epoch 4/200
6/6 - 0s - loss: 0.3607 - accuracy: 0.8554 - val_loss: 0.4998 - val_accuracy: 0.8571 - 55ms/epoch - 9ms/step
Epoch 5/200
6/6 - 0s - loss: 0.3706 - accuracy: 0.8614 - val_loss: 0.4469 - val_accuracy: 0.8571 - 51ms/epoch - 8ms/step
Epoch 6/200
6/6 - 0s - loss: 0.3370 - accuracy: 0.8795 - val_loss: 0.4152 - val_accuracy: 0.8571 - 55ms/epoch - 9ms/step
Epoch 7/200
6/6 - 0s - loss: 0.3072 - accuracy: 0.8795 - val_loss: 0.3921 - val_accuracy: 0.8810 - 49ms/epoch - 8ms/step
Epoch 8/200
6/6 - 0s - loss: 0.2985 - accuracy: 0.9096 - val_loss: 0.3674 - val_accuracy: 0.9048 - 66ms/epoch - 11ms/step
Epoch 9/200
6/6 - 0s - loss: 0

In [ ]:
# Evaluate model on test set
hyper_df = evaluate_model(h_model, X_test, Y_test)

# Set index to hypertuned
hyper_df.index = ["Hypertuned"]

# Append results in dataframe
results.append(hyper_df)

3/3 [==============================] - 0s 5ms/step - loss: 0.2696 - accuracy: 0.8734


,loss,accuracy
Baseline,0.264184,0.898734
Hypertuned,0.269589,0.873418


## Hyperparameter Tuning II

In [ ]:
! pip install keras-tuner

In [ ]:
# https://towardsdatascience.com/hyperparameter-tuning-with-kerastuner-and-tensorflow-c4a4d690b31a

import tensorflow as tf
import kerastuner as kt
from tensorflow import keras
print(f"TensorFlow Version: {tf.__version__}")
print(f"KerasTuner Version: {kt.__version__}")

TensorFlow Version: 2.9.2
KerasTuner Version: 1.0.3


In [ ]:
# Load and split data into train and test sets
X_train = x_train
Y_train = y_train
X_test  = x_test
Y_test  = y_test

# Normalize pixels to values between 0 and 1
#X_train = X_train.astype('float32') / 255.0
#X_test = X_test.astype('float32') / 255.0

In [ ]:
# Build baseline model with Sequential API

b_model = Sequential()

b_model.add(Dense(units = 100, input_dim = x_train.shape[1], kernel_initializer = 'glorot_uniform'))
b_model.add(BatchNormalization())
b_model.add(Activation('tanh')) # Hyperbolic tangent activation function.
b_model.add(Dropout(0.3))

b_model.add(Dense(units = 10,kernel_initializer = 'glorot_uniform'))
b_model.add(BatchNormalization())
b_model.add(Activation('tanh'))
b_model.add(Dropout(0.2))

b_model.add(Dense(units = 10,kernel_initializer = 'glorot_uniform'))
b_model.add(BatchNormalization())
b_model.add(Activation('tanh'))
b_model.add(Dropout(0.1))

b_model.add(Dense(units = 1,kernel_initializer = 'he_uniform'))  
b_model.add(Activation('sigmoid'))         

In [ ]:
# Set training parameters
b_model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                         optimizer = 'nadam',
                         metrics = ['accuracy'])

In [ ]:
# Number of epochs
NUM_EPOCHS = 200

# Early stopping set after 20 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# Train model
b_model.fit(X_train, Y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

In [ ]:
import pandas as pd

def evaluate_model(model, X_test, Y_test):
    """
    evaluate model on test set and show results in dataframe.
    
    Parameters
    ----------
    model : keras model
        trained keras model.
    X_test : numpy array
        Features of holdout set.
    y_test : numpy array
        Labels of holdout set.
        
    Returns
    -------
    display_df : DataFrame
        Pandas dataframe containing evaluation results.
    """
    eval_dict = model.evaluate(X_test, Y_test, return_dict=True)
    
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    return display_df

# Evaluate model on test set and add results to dataframe
results = evaluate_model(b_model, X_test, Y_test)

# Set index to 'Baseline'
results.index = ['Baseline']

# Display results
results.head()

3/3 [==============================] - 0s 6ms/step - loss: 0.4586 - accuracy: 0.8354


,loss,accuracy
Baseline,0.458599,0.835443


In [ ]:
def build_model(hp,X_train=X_train):
    """
    Builds model and sets up hyperparameter space to search.
    
    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.
        
    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    # Initialize sequential API and start building model.
    model = keras.Sequential()
    model.add(Dense(units = 100, input_dim = X_train.shape[1], kernel_initializer = 'glorot_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.3))
    
    # Tune the number of hidden layers and units in each.
    # Number of hidden layers: 1 - 6
    # Number of Units: 8 - 14 with stepsize of 1
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=20, max_value=40, step=2),
                kernel_initializer = 'glorot_uniform',
                activation="tanh")
            )
        model.add(BatchNormalization())
        # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(keras.layers.Dense(units=1, activation="sigmoid",kernel_initializer = 'he_uniform'))
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.BinaryCrossentropy(),
                  metrics=["accuracy"])
    
    return model

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=NUM_EPOCHS,
                     factor=3,
                     hyperband_iterations=1,
                     directory="kt_dir",
                     project_name="kt_hyperband")

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 20, 'max_value': 40, 'step': 2, 'sampling': None}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

tuner.search(X_train, Y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

Trial 254 Complete [00h 00m 07s]
val_accuracy: 0.8809523582458496

Best val_accuracy So Far: 0.976190447807312
Total elapsed time: 00h 14m 15s


In [ ]:
for h_param in [f"units_{i}" for i in range(1,4)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

units_1 38
units_2 24
units_3 38
learning_rate 0.0001


In [ ]:
# Get the optimal hyperparameters from the results
best_hps = tuner.get_best_hyperparameters()[0]
best_hps

In [ ]:
# Build model
h_model = tuner.hypermodel.build(best_hps)
h_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 100)               262300    
                                                                 
 batch_normalization_4 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 100)               0         
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 38)                3838      
                                                                 
 batch_normalization_5 (Batc  (None, 38)               152       
 hNormalization)                                      

In [ ]:
# Train the hypertuned model
h_model.fit(X_train, Y_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early], verbose=2)

Epoch 1/200
6/6 - 2s - loss: 0.7369 - accuracy: 0.5904 - val_loss: 0.6465 - val_accuracy: 0.6667 - 2s/epoch - 269ms/step
Epoch 2/200
6/6 - 0s - loss: 0.4496 - accuracy: 0.7952 - val_loss: 0.5402 - val_accuracy: 0.8333 - 62ms/epoch - 10ms/step
Epoch 3/200
6/6 - 0s - loss: 0.4149 - accuracy: 0.8133 - val_loss: 0.4574 - val_accuracy: 0.8571 - 66ms/epoch - 11ms/step
Epoch 4/200
6/6 - 0s - loss: 0.3696 - accuracy: 0.8554 - val_loss: 0.4283 - val_accuracy: 0.8571 - 62ms/epoch - 10ms/step
Epoch 5/200
6/6 - 0s - loss: 0.3169 - accuracy: 0.8735 - val_loss: 0.4129 - val_accuracy: 0.8571 - 61ms/epoch - 10ms/step
Epoch 6/200
6/6 - 0s - loss: 0.3149 - accuracy: 0.8795 - val_loss: 0.3839 - val_accuracy: 0.8571 - 64ms/epoch - 11ms/step
Epoch 7/200
6/6 - 0s - loss: 0.2579 - accuracy: 0.9036 - val_loss: 0.3736 - val_accuracy: 0.8571 - 60ms/epoch - 10ms/step
Epoch 8/200
6/6 - 0s - loss: 0.2589 - accuracy: 0.8795 - val_loss: 0.3866 - val_accuracy: 0.8571 - 62ms/epoch - 10ms/step
Epoch 9/200
6/6 - 0s - lo

In [ ]:
# Evaluate model on test set
hyper_df = evaluate_model(h_model, X_test, Y_test)

3/3 [==============================] - 0s 6ms/step - loss: 0.2706 - accuracy: 0.8861


In [ ]:
# Set index to hypertuned
hyper_df.index = ["Hypertuned"]

In [ ]:
# Append results in dataframe
results.append(hyper_df)

,loss,accuracy
Baseline,0.458599,0.835443
Hypertuned,0.270624,0.886076
